In [1]:
# https://adventofcode.com/2018/day/21

struct Instruction
    op::String
    params::Array{Int}

    function Instruction(line::String)
        caps = match(r"([a-z]+)\s(\d+)\s(\d+)\s(\d+)", line).captures
        new(caps[1], parse.(Int, caps[2:end]))
    end
end

ip, program = open("21.txt", "r") do file
    ip = parse(Int, match(r"#ip (\d+)", readline(file)).captures[1])
    
    program = readlines(file) .|> Instruction

    ip, program
end


(1, Instruction[Instruction("seti", [123, 0, 5]), Instruction("bani", [5, 456, 5]), Instruction("eqri", [5, 72, 5]), Instruction("addr", [5, 1, 1]), Instruction("seti", [0, 0, 1]), Instruction("seti", [0, 6, 5]), Instruction("bori", [5, 65536, 4]), Instruction("seti", [13431073, 4, 5]), Instruction("bani", [4, 255, 3]), Instruction("addr", [5, 3, 5])  …  Instruction("addr", [2, 1, 1]), Instruction("addi", [1, 1, 1]), Instruction("seti", [25, 4, 1]), Instruction("addi", [3, 1, 3]), Instruction("seti", [17, 8, 1]), Instruction("setr", [3, 4, 4]), Instruction("seti", [7, 7, 1]), Instruction("eqrr", [5, 0, 3]), Instruction("addr", [3, 1, 1]), Instruction("seti", [5, 9, 1])])

In [2]:
# the ops

addr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] + r[b + 1]; r)
addi(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] + b; r)
mulr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] * r[b + 1]; r)
muli(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] * b; r)
banr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] & r[b + 1]; r)
bani(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] & b; r)
borr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] | r[b + 1]; r)
bori(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] | b; r)
setr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1]; r)
seti(r, a, b, c) = (r = copy(r); r[c + 1] = a; r)
gtir(r, a, b, c) = (r = copy(r); r[c + 1] = a > r[b + 1]; r)
gtri(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] > b; r)
gtrr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] > r[b + 1]; r)
eqir(r, a, b, c) = (r = copy(r); r[c + 1] = a == r[b + 1]; r)
eqri(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] == b; r)
eqrr(r, a, b, c) = (r = copy(r); r[c + 1] = r[a + 1] == r[b + 1]; r)


eqrr (generic function with 1 method)

In [93]:
# part 1

registers = zeros(Int, 6)
while registers[ip + 1] + 1 in 1:length(program)
    instruction = program[registers[ip + 1] + 1]
    op = getfield(Main, Symbol(instruction.op))
    registers = op(registers, instruction.params...)
    registers[ip + 1] += 1
end

registers[1]


1248

In [9]:
# part 1, disassembly

a = 3115806   # through simple simulation (first f where e < 256)

f = 123
while f != 72
    f &= 456
end

f = 0
while true
    e = f | 65536
    f = 13431073
    while true
        d = e & 255
        f += d
        f &= 16777215
        f *= 65899
        f &= 16777215
        d = 256 > e
        if d
            if f == a
                return
            else
                break
            end
        end

        d = 0
        while true
            c = d + 1
            c *= 256
            c = c > e
            if c
                break
            end
            d += 1
        end

        e = d
    end
end

a


3115806

In [30]:
# part 2

fs = Set()
e = 0
f = 0
pf = 0
while true
    e = f | 65536
    f = 13431073
    while true
        f += e & 255
        f &= 16777215
        f *= 65899
        f &= 16777215
        if e < 256 
            # find the f that creates the first cycle
            # setting a to the previous f will run the longest
            if f in fs
                return pf
            end
            push!(fs, f)
            pf = f
            break
        end
        
        # this replaces the very slow division in part 1
        e = div(e, 256)
    end
end


13959373